In [1]:
# Import packages
import os, time, json
import tmdbsimple as tmdb 
import pandas as pd
from tqdm.notebook import tqdm_notebook
# Create the folder for saving files (if it doesn't exist)
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)


['.ipynb_checkpoints',
 'cleaned-basics.csv',
 'cleaned-ratings.csv',
 'Movies-ERD.png',
 'Project2_Part2.sql',
 'Project2_Part2_mySQL_Model.mwb',
 'title-akas-us-only.csv',
 'title.basics.tsv.gz',
 'title.ratings.tsv.gz']

In [2]:
def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)


In [3]:
# Load in the dataframe from project part 1 as basics:
basics = pd.read_csv('Data/cleaned-basics.csv')



In [4]:
YEAR = 2010



In [5]:
errors = [ ]



In [6]:
#Defining the JSON file to store results for year
JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'


In [7]:
# Check if file exists
file_exists = os.path.isfile(JSON_FILE)


In [9]:
# If it does not exist: create it
if file_exists == False:
    # Print a message indicating the file is being created 
    print(f"Creating {JSON_FILE} for API results for {YEAR}.")
    # save an empty dict with just "imdb_id" to the new json file.
    with open(JSON_FILE,'w') as f:
        json.dump([{'imdb_id':0}],f)
else: 
    print(f"The file {JSON_FILE} already exists.")

Creating Data/tmdb_api_results_2010.json for API results for 2010.


In [10]:
#Saving new year as the current df
df = basics.loc[ basics['startYear'] == YEAR].copy()
# saving movie ids to separate variable
movie_ids = df['tconst']



In [11]:
# Load existing data from json into a dataframe called "previous_df"
previous_df = pd.read_json(JSON_FILE)



In [12]:
# filter out any ids that are already in the JSON_FILE
movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]



In [15]:
# Loop through movie_ids_to_get with a tqdm progress bar
for movie_id in tqdm_notebook(movie_ids_to_get, f"Movies from {YEAR}"):
    #Get index and movie id from list
    try:
        # Retrieve then data for the movie id
        temp = get_movie_with_rating(movie_id)  
        # Append/extend results to existing file using a pre-made function
        write_json(temp,JSON_FILE)
        # Short 20 ms sleep to prevent overwhelming server
        time.sleep(0.02)
    
    except Exception as e:
        errors.append([movie_id, e])


Movies from 2010:   0%|          | 0/3862 [00:00<?, ?it/s]

In [17]:
print(f"- Total errors: {len(errors)}")



- Total errors: 3863


In [18]:
final_year_df = pd.read_json(JSON_FILE)
final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)


In [ ]:
# Instead of previous_df=pd.read_json:
#previous_df = read_and_fix_json(JSON_FILE)

